## Building Technical Strategies with VectorBT

In [1]:
!pip install vectorbt

In [2]:
import pandas as pd
import vectorbt as vbt
from IPython.display import Markdown, display

Define the start and end dates for data download

In [3]:
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"

Download historical closing prices for specified symbols from Yahoo Finance

In [4]:
prices = vbt.YFData.download(
    ["META", "AAPL", "AMZN", "NFLX", "GOOG"], start=start, end=end
).get("Close")

Calculate fast and slow moving averages

In [5]:
fast_ma = vbt.MA.run(prices, 10, short_name="fast")
slow_ma = vbt.MA.run(prices, 30, short_name="slow")

Generate entry signals when the fast MA crosses above the slow MA

In [6]:
entries = fast_ma.ma_crossed_above(slow_ma)
display(entries)

fast_window                   10                            
slow_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
2016-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2019-12-24 05:00:00+00:00  False  False  False  False  False
2019-12-26 05:00:00+00:00  False  False  False  False  False
2019-12-27 05:00:00+00:00  False  False  False  False  False
2019-12-30 05:00:00+00:00  False  False  False  False  False
2019-12-31 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

Generate exit signals when the fast MA crosses below the slow MA

In [7]:
exits = fast_ma.ma_crossed_below(slow_ma)
display(exits)

fast_window                   10                            
slow_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
2016-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2019-12-24 05:00:00+00:00  False  False  False  False  False
2019-12-26 05:00:00+00:00  False  False  False  False  False
2019-12-27 05:00:00+00:00  False  False  False  False  False
2019-12-30 05:00:00+00:00  False  False  False  False  False
2019-12-31 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

Create a portfolio using the generated entry and exit signals

In [8]:
pf = vbt.Portfolio.from_signals(prices, entries, exits, freq="1d")

Display portfolio statistics

In [9]:
display(pf.stats())

/var/folders/pp/g90q205n5_39x714vvq7pp_00000gn/T/ipykernel_16224/4100483198.py:1: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x122063880>. Pass column to select a single column/group.
  display(pf.stats())


Start                          2016-01-04 05:00:00+00:00
End                            2019-12-31 05:00:00+00:00
Period                                1006 days 00:00:00
Start Value                                        100.0
End Value                                     176.376713
Total Return [%]                               76.376713
Benchmark Return [%]                          152.731367
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               25.869036
Max Drawdown Duration                  347 days 00:00:00
Total Trades                                        17.4
Total Closed Trades                                 16.4
Total Open Trades                                    1.0
Open Trade PnL                                 23.097864
Win Rate [%]                                    49.79085
Best Trade [%]                                  27.62886
Worst Trade [%]                

Plot the total return of the portfolio for each symbol

In [10]:
(pf.total_return().groupby("symbol").mean().vbt.barplot())

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '76e55ece-d90b-40e7-bb45-e47bf3e4f750',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.7348317 , 1.28553102, 0.50135305, 0.16619115, 0.13092872])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

Create a portfolio assuming buy-and-hold strategy and plot the total return

In [11]:
(
    vbt.Portfolio.from_holding(prices, freq="1d")
    .total_return()
    .groupby("symbol")
    .mean()
    .vbt.barplot()
)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'c350ef2d-0f06-4f34-b90f-9f7aefde228d',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.98283284, 1.90089315, 0.80230255, 1.00792421, 1.94261559])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

Split the prices data into 4 equal parts

In [12]:
mult_prices, _ = prices.vbt.range_split(n=4)

Calculate fast and slow moving averages for each split

In [13]:
fast_ma = vbt.MA.run(mult_prices, [10, 20], short_name="fast")
slow_ma = vbt.MA.run(mult_prices, [30, 30], short_name="slow")

Generate entry and exit signals for each split

In [14]:
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

Create a portfolio using the generated entry and exit signals for each split

In [15]:
pf = vbt.Portfolio.from_signals(mult_prices, entries, exits, freq="1D")

Plot the total return of the portfolio for each split and symbol

In [16]:
(
    pf.total_return()
    .groupby(["split_idx", "symbol"])
    .mean()
    .unstack(level=-1)
    .vbt.barplot()
)

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'bar',
              'uid': 'bc8633fd-7e6d-40ff-8039-3519f4bbd2b3',
              'x': array([0, 1, 2, 3]),
              'y': array([0.05063227, 0.11321568, 0.10540665, 0.44102591])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'bar',
              'uid': '780f2e93-aaaf-43c8-8d01-1e2d317cfaed',
              'x': array([0, 1, 2, 3]),
              'y': array([0.30121359, 0.10976334, 0.26265571, 0.01039871])},
             {'name': 'GOOG',
              'showlegend': True,
              'type': 'bar',
              'uid': '83d50fd1-0b8c-461e-9da0-c314dec67b11',
              'x': array([0, 1, 2, 3]),
              'y': array([-0.08226519,  0.17835176,  0.01148375,  0.13153614])},
             {'name': 'META',
              'showlegend': True,
              'type': 'bar',
              'uid': '4b1aeba2-c7d0-4e3e-b251-dcd00d3a02bd',
              

Display order statistics

In [17]:
display(pf.orders.stats(group_by=True))

Start                                0
End                                250
Period               251 days 00:00:00
Total Records                      291
Total Buy Orders                   157
Total Sell Orders                  134
Min Size                      0.253075
Max Size                      4.706514
Avg Size                      1.634929
Avg Buy Size                  1.643704
Avg Sell Size                 1.624647
Avg Buy Price               103.313656
Avg Sell Price              104.434968
Total Fees                         0.0
Min Fees                           0.0
Max Fees                           0.0
Avg Fees                           0.0
Avg Buy Fees                       0.0
Avg Sell Fees                      0.0
Name: group, dtype: object

Display the Sharpe ratio of the portfolio

In [18]:
display(pf.sharpe_ratio())

fast_window  slow_window  split_idx  symbol
10           30           0          META     -0.472595
                                     AAPL      1.125234
                                     AMZN      1.633329
                                     NFLX     -0.470585
                                     GOOG     -0.051112
                          1          META      0.449941
                                     AAPL      1.043138
                                     AMZN      0.611364
                                     NFLX     -0.359590
                                     GOOG      1.754605
                          2          META     -0.572691
                                     AAPL      0.049780
                                     AMZN      1.255488
                                     NFLX     -0.171628
                                     GOOG      0.088257
                          3          META      0.556393
                                     AAPL      2.921360
    

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.